In [1]:
filename = 'ibm02.hgr'
with open(f'./data/{filename}', 'r') as f:
    lines = f.readlines()
    num_nets, num_nodes = map(int, lines[0].split())
    hypergraph_vertices = list(range(num_nodes))
    hypergraph_edges = []
    for line in lines[1:]:
        hypergraph_edges.append([int(node) - 1 for node in line.split()])
print(num_nets, num_nodes)

19584 19601


In [2]:
from scipy.sparse import coo_matrix

row = []
col = []
value = []
for i, e in enumerate(hypergraph_edges):
    for v in e:
        row.append(v)
        col.append(i)
        value.append(1)
H = coo_matrix((value, (row, col)), shape=(num_nodes, num_nets), dtype=float)
print(H.shape)

(19601, 19584)


In [3]:
from scipy.sparse.linalg import svds
from utils import normalize_hypergraph_incidence_matrix
import numpy as np

H = normalize_hypergraph_incidence_matrix(H)
U, S, Vt = svds(H, k=2, which='LM', random_state=42, solver='propack', maxiter=10000)
U = U[:, np.argsort(S)[::-1]]
for i in range(U.shape[1]):    
    if U[np.argmax(np.absolute(U[:,i])),i] < 0:
        U[:,i] = -U[:,i]
print(U.shape, S.shape, Vt.shape)
print(S)

(19601, 2) (2,) (2, 19584)
[0.99715396 1.        ]


In [4]:
import torch

# Create the hyperedge_index tensor
hyperedge_index = torch.tensor(np.array([
    np.concatenate(hypergraph_edges),
    np.repeat(np.arange(len(hypergraph_edges)), [len(e) for e in hypergraph_edges])
]), dtype=torch.long)

print(hyperedge_index.shape)

torch.Size([2, 81199])


In [5]:
from utils import create_clique_expansion_graph, compute_topological_features, create_partition_id_feature

adj_matrix, node_degree, pin_count = create_clique_expansion_graph(hypergraph_vertices, hypergraph_edges)
clique_topo_features = compute_topological_features(adj_matrix, 2, True, False)
star_topo_features = torch.tensor(U.copy(), dtype=torch.float)
partition_feature = create_partition_id_feature(len(hypergraph_vertices), filename)
id = np.arange(len(hypergraph_vertices))
features = np.column_stack([clique_topo_features, star_topo_features, node_degree, pin_count, partition_feature, id])
del adj_matrix, node_degree, pin_count, clique_topo_features, star_topo_features, id
deg_feature_norm = np.linalg.norm(features[:, 4])
features[:, 0] = features[:, 0] / np.linalg.norm(features[:, 0]) * deg_feature_norm
features[:, 1] = features[:, 1] / np.linalg.norm(features[:, 1]) * deg_feature_norm
features[:, 2] = features[:, 2] / np.linalg.norm(features[:, 2]) * deg_feature_norm
features[:, 3] = features[:, 3] / np.linalg.norm(features[:, 3]) * deg_feature_norm
# features[:, 4] = features[:, 4] / np.linalg.norm(features[:, 4]) * deg_feature_norm
features[:, 5] = features[:, 5] / np.linalg.norm(features[:, 5]) * deg_feature_norm
features[:, 6] = features[:, 6] / np.linalg.norm(features[:, 6]) * deg_feature_norm
features[:, 7] = features[:, 7] / np.linalg.norm(features[:, 7]) * deg_feature_norm
print(features.shape)

(19601, 8)


In [6]:
from models import HyperData

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# features = features[:, :7]
x = torch.tensor(features, dtype=torch.float)
data = HyperData(x=x, hyperedge_index=hyperedge_index)
data = data.to(device)
W = torch.sparse_coo_tensor(hyperedge_index, torch.ones(hyperedge_index.shape[1]), (num_nodes, num_nets)).to(device)
D = torch.sparse.sum(W, dim=1).to_dense().unsqueeze(1)
print(device)
print(W.shape)
print(D.shape)
print(data.x.shape)

cuda
torch.Size([19601, 19584])
torch.Size([19601, 1])
torch.Size([19601, 8])


In [7]:
from models import GraphPartitionModel

input_dim = 8
latent_dim = 64
hidden_dim = 256
num_partitions = 2
num_epochs = 1000
model = GraphPartitionModel(input_dim, hidden_dim, latent_dim, num_partitions, True)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
model.train()

for epoch in range(num_epochs):
    optimizer.zero_grad()
    Y = model(data)
    loss, kl_loss, hyperedge_cut_loss, balance_loss = model.combined_loss(Y, W, D, alpha=0.0005, beta=5, gamma=2.5)
    loss.backward()
    # torch.nn.utils.clip_grad_norm_(model.parameters(), 3)
    optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}, KL Loss: {kl_loss.item()}, Ncut Loss: {hyperedge_cut_loss.item()}, Balance Loss: {balance_loss.item()}')

Epoch 1, Loss: 3.4528467655181885, KL Loss: 126.15348052978516, Ncut Loss: 0.661339282989502, Balance Loss: 0.033229485154151917
Epoch 2, Loss: 3.4179859161376953, KL Loss: 110.77044677734375, Ncut Loss: 0.6605503559112549, Balance Loss: 0.02393956109881401
Epoch 3, Loss: 3.3904411792755127, KL Loss: 99.36753845214844, Ncut Loss: 0.6598483920097351, Balance Loss: 0.016606107354164124
Epoch 4, Loss: 3.3709723949432373, KL Loss: 90.67166900634766, Ncut Loss: 0.6594382524490356, Balance Loss: 0.01137823611497879
Epoch 5, Loss: 3.353639841079712, KL Loss: 83.72590637207031, Ncut Loss: 0.6589940190315247, Balance Loss: 0.006722700782120228
Epoch 6, Loss: 3.3426928520202637, KL Loss: 78.05967712402344, Ncut Loss: 0.6587469577789307, Balance Loss: 0.003971287980675697
Epoch 7, Loss: 3.3340511322021484, KL Loss: 73.27456665039062, Ncut Loss: 0.6585678458213806, Balance Loss: 0.0018298563081771135
Epoch 8, Loss: 3.328763961791992, KL Loss: 69.01258087158203, Ncut Loss: 0.6584699749946594, Balan

In [8]:
Y = model(data)
print(model.hyperedge_cut_loss(Y, W).item())

459.51171875


In [9]:
from utils import evalPoint, evaluate_partition
import time
import multiprocessing as mp

model.eval()
best_cut, best_imbalance = evaluate_partition(partition_feature, hypergraph_vertices, hypergraph_edges, num_partitions)
best_partition_id = partition_feature
reason_time = 0
vcycle_time = 0
for tau in range(11):
    tau_best_cut = float('inf')
    t0 = time.time()
    partitions = model.sample(data, m=12)
    t1 = time.time()
    reason_time += t1 - t0
    t0 = time.time()
    processes = []
    pool = mp.Pool(processes=6)
    for m in range(len(partitions)):
        processes.append(pool.apply_async(evalPoint, (m, partitions[m], hypergraph_vertices, hypergraph_edges, num_partitions, filename, False, False)))
        time.sleep(0.01)
    pool.close()
    pool.join()
    for process in processes:
        cut, imbalance, partition_id = process.get()
        tau_best_cut = min(tau_best_cut, cut)
        if cut < best_cut:
            best_cut = cut
            best_imbalance = imbalance
            best_partition_id = partition_id
    t1 = time.time()
    vcycle_time += t1 - t0
    print(f'Tau: {tau}, Best Cut: {best_cut}, Imbalance: {best_imbalance:.3f}, Cut: {tau_best_cut}')
    best_partition_id = best_partition_id / np.linalg.norm(best_partition_id) * np.linalg.norm(data.x[:, 4].cpu().numpy())
    data.x[:, 6] = torch.tensor(best_partition_id, dtype=torch.float).to(device)
print(f'Final Best Cut: {best_cut}, Imbalance: {best_imbalance:.3f}, Reasoning Time: {reason_time:.3f}, V-Cycle Time: {vcycle_time:.3f}')

Tau: 0, Best Cut: 376, Imbalance: 0.010, Cut: 417
Tau: 1, Best Cut: 376, Imbalance: 0.010, Cut: 418
Tau: 2, Best Cut: 376, Imbalance: 0.010, Cut: 418
Tau: 3, Best Cut: 376, Imbalance: 0.010, Cut: 419
Tau: 4, Best Cut: 376, Imbalance: 0.010, Cut: 419
Tau: 5, Best Cut: 376, Imbalance: 0.010, Cut: 417
Tau: 6, Best Cut: 376, Imbalance: 0.010, Cut: 417
Tau: 7, Best Cut: 376, Imbalance: 0.010, Cut: 419
Tau: 8, Best Cut: 376, Imbalance: 0.010, Cut: 417
Tau: 9, Best Cut: 376, Imbalance: 0.010, Cut: 419
Tau: 10, Best Cut: 376, Imbalance: 0.010, Cut: 419
Final Best Cut: 376, Imbalance: 0.010, Reasoning Time: 0.154, V-Cycle Time: 18.471
